### Задание 4
(опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.

In [1]:
import pandas as pd

# news
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [2]:
# users
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [3]:
# vectorize news

#from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

#предобработка текстов
import re
import numpy as np
import nltk

# download resource
nltk.download('stopwords')

from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

stopword_ru = stopwords.words('russian')

morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maximdoroshenko/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# additional stopwords for quality improve

with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [5]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [6]:
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)
news['title']

<ipython-input-5-7ee348d9b386>:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


0        [заместитель, председатель, правительство, рф,...
1        [матч, финал, кубок, россия, футбол, приостано...
2        [форвард, авангард, томаш, заборский, прокомме...
3        [главный, тренер, кубань, юрий, красножанин, п...
4        [решение, попечительский, совет, владивостокск...
                               ...                        
26995    [учёный, токийский, университет, морской, наук...
26996    [глава, кафедра, отечественный, история, xx, в...
26997    [американский, учёный, уточнить, возраст, расп...
26998    [последний, тропический, углеродный, цикл, чув...
26999    [жить, примерно, тыс, назад, территория, совре...
Name: title, Length: 27000, dtype: object

In [7]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [15]:
# TF-IDF

from gensim.models import TfidfModel

tfidf = TfidfModel(common_corpus)
corpus_tfidf = tfidf[common_corpus]

In [19]:
from gensim.models import LdaModel

# train
lda_tfidf = LdaModel(corpus_tfidf, num_topics=25, id2word=common_dictionary)#, passes=10)idf = LdaModel(common_corcorpus_tfidfpus, num_topics=25, id2word=common_dictionary)#, passes=10)

In [21]:
# save
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda_tfidf")
lda_tfidf.save(temp_file)

# Load a potentially pretrained model from disk.
lda_tfidf = LdaModel.load(temp_file)

In [22]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda_tfidf[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(4, 0.028880106),
 (8, 0.35681397),
 (10, 0.102220796),
 (16, 0.4033433),
 (21, 0.03179324),
 (22, 0.054548237)]

In [19]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: рост всё экономика рынок жизнь проект новый
topic_1: nn станция университет nnn армия академия газета
topic_2: путин владимир тема песок пресссекретарить запуск лауреат
topic_3: температура завод градус погода москва билет вино
topic_4: газ турция турецкий администрация франция лодка французский
topic_5: сша американский военный поверхность штат россия испытание
topic_6: гражданин болезнь космос топливо глава активность введение
topic_7: автор товар индия сопровождать концерн сооружение корь
topic_8: ребёнок земля всё область первый регион население
topic_9: мозг кровь выяснить эксперимент сотрудник задержать северный
topic_10: погибнуть операция решение банк nn произойти рф
topic_11: цена мужчина nn проверка планета общество тыс
topic_12: снижение сократиться золото аналог статистика золотой соотношение
topic_13: млн место рейтинг великобритания nn первый составить
topic_14: статья пациент лечение помощь исследование катастрофа проживать
topic_15: млрд рубль банк исследование

In [23]:
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda_tfidf[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [24]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0,0.0,0.0,0.000000,0.017783,0.0,0.037926,0.0,0.181210,...,0.012006,0.434175,0.000000,0.0,0.0,0.000000,0.087110,0.072143,0.0,0.0
1,4896,0.0,0.0,0.0,0.118248,0.000000,0.0,0.000000,0.0,0.056014,...,0.000000,0.799811,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,4897,0.0,0.0,0.0,0.000000,0.028880,0.0,0.000000,0.0,0.356622,...,0.000000,0.403541,0.000000,0.0,0.0,0.000000,0.031793,0.054613,0.0,0.0
3,4898,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.380844,...,0.000000,0.491655,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
4,4899,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.266630,...,0.000000,0.503604,0.034505,0.0,0.0,0.033812,0.000000,0.000000,0.0,0.0


In [25]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [26]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [27]:
user_articles_list = users['articles'].iloc[33]

def get_user_embedding(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.mean(user_vector, 0)
    return user_vector

In [28]:
get_user_embedding(user_articles_list)

array([0.        , 0.        , 0.05363805, 0.        , 0.01683263,
       0.        , 0.05730499, 0.        , 0.19531964, 0.        ,
       0.03379096, 0.00599162, 0.        , 0.        , 0.02304116,
       0.00619374, 0.46608179, 0.00213268, 0.03432373, 0.        ,
       0.02419515, 0.03112794, 0.03434957, 0.00224391, 0.        ])

In [29]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.006423,0.0,0.003283,0.018374,0.015932,0.0,0.047000,0.0,0.228023,...,0.012720,0.456344,0.000000,0.007215,0.004758,0.006044,0.023430,0.048970,0.019616,0.000000
1,u108690,0.000000,0.0,0.025044,0.004220,0.008087,0.0,0.078936,0.0,0.268349,...,0.005787,0.381456,0.002272,0.004602,0.001940,0.017472,0.029108,0.053839,0.002397,0.002266
2,u108339,0.000000,0.0,0.035758,0.012647,0.009124,0.0,0.072640,0.0,0.269220,...,0.014578,0.364731,0.001797,0.013509,0.002250,0.023064,0.018421,0.054164,0.002418,0.000000


In [30]:
# users churn prediction

target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [31]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,churn
0,u105138,0.006423,0.0,0.003283,0.018374,0.015932,0.0,0.047000,0.0,0.228023,...,0.456344,0.000000,0.007215,0.004758,0.006044,0.023430,0.048970,0.019616,0.000000,0
1,u108690,0.000000,0.0,0.025044,0.004220,0.008087,0.0,0.078936,0.0,0.268349,...,0.381456,0.002272,0.004602,0.001940,0.017472,0.029108,0.053839,0.002397,0.002266,1
2,u108339,0.000000,0.0,0.035758,0.012647,0.009124,0.0,0.072640,0.0,0.269220,...,0.364731,0.001797,0.013509,0.002250,0.023064,0.018421,0.054164,0.002418,0.000000,1


In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [34]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression()

In [35]:
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

array([0.11131315, 0.04711085, 0.26939625, 0.17392922, 0.24711805,
       0.17161291, 0.11096639, 0.08686035, 0.05239288, 0.09147104])

In [42]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
import itertools

In [60]:
def calc_fscore(precision, recall):
    result = []
    for i in range(len(precision)):
        if precision[i] + recall[i] == 0:
            result.append(0)
        else:
            result.append(((2 * precision[i] * recall[i]) / (precision[i] + recall[i])))
    return result

In [61]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = calc_fscore(precision, recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.180312, F-Score=0.409, Precision=0.311, Recall=0.600


In [62]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.8099191813477528